In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Object detection with few shot learning

This notebook demonstrates how to detect a trasformer installed on a pole from Imagery Insights using the Gemini 2.5 Flash model via Vertex AI.

## Install Required Libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery google-genai

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID and Region.

In [ ]:
PROJECT_ID = 'sarthaks-lab'  # @param {type:"string"}
REGION = 'us-central1'      # @param {type:"string"}
MODEL_ID = "gemini-2.5-flash" # @param {type:"string"}
BIGQUERY_DATASET_ID = 'imagery_insights___preview___us' # @param {type:"string"}
BIGQUERY_TABLE = "latest_observations" # @param {type:"string"}
ASSET_TYPE = "ASSET_CLASS_UTILITY_POLE" # @param {type:"string"}
LIMIT = 5 # @param {type:"integer"}

## Imports and Vertex AI Initialization

In [ ]:
import vertexai
from google.cloud import bigquery
from google import genai
from google.genai.types import Content, Part

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=REGION)

## Fetch Image URIs from BigQuery

Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify.

In [ ]:
BIGQUERY_SQL_QUERY = f"""
SELECT
  *
FROM
  `{PROJECT_ID}.{BIGQUERY_DATASET_ID}.{BIGQUERY_TABLE}`
   WHERE asset_type = "{ASSET_TYPE}"
LIMIT {LIMIT};
"""

# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]

    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Define Image Classification Function

This function takes a GCS URI and a prompt, then uses the Gemini 2.5 Flash model to generate a description of the image.

In [ ]:
def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:
    """
    Classifies an image using the Gemini 2.5 Flash model by directly passing its GCS URI.
    """
    try:
        contents = [
            prompt,
            Part(file_data={'file_uri': gcs_uri, 'mime_type': 'image/jpeg'})
        ]
        response = client.models.generate_content(model=MODEL_ID, contents=contents)
        return response.text
    except Exception as e:
        print(f"Error classifying image from URI {gcs_uri}: {e}")
        return "Classification failed."

## Classify Images

Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt.

In [ ]:
few_shot_uri_with_transformer = "gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/ef8ed4fa-a74e-4179-a0d1-8f091caa71bb/v0/o1:54882fdccd7682a5935d39470e081c8b:00250082.jpg" # @param {type:"string"}
few_shot_json_with_transformer = """{  "type": "utility pole",  "transformers": 1,  "additional_notes": ""}""" # @param {type:"string"}

few_shot_uri_no_attachments = "gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/ef8ed4fa-a74e-4179-a0d1-8f091caa71bb/v0/o1:050b9e2dcf5d6a97c28eceb7c9b819c2:00250082.jpg" # @param {type:"string"}
few_shot_json_no_attachments = """{  "type": "utility pole",  "transformers": 0,  "additional_notes": ""}""" # @param {type:"string"}

print("Few-shot example URIs and JSON outputs defined.")

Few-shot example URIs and JSON outputs defined.


In [ ]:
few_shot_json_with_transformer = """{
  "type": "utility pole",
  "transformers": 1,
  "telephone_or_junction_boxes": 0,
  "additional_notes": ""
}"""

few_shot_json_no_attachments = """{
  "type": "utility pole",
  "transformers": 0,
  "telephone_or_junction_boxes": 0,
  "additional_notes": ""
}"""

prompt = f"""You will be provided with a photo of a utility pole:
{{photo_of_utility_pole}}

Here are some examples:
Example 1 (Utility pole with a transformer):
{few_shot_uri_with_transformer}
Expected Output:
```json
{few_shot_json_with_transformer}
```

Example 2 (Utility pole without attachments):
{few_shot_uri_no_attachments}
Expected Output:
```json
{few_shot_json_no_attachments}
```

Instructions:

1. Analyze the provided image. If the image does not clearly show a utility pole, return: {{"error": "No utility pole detected in the image."}}
2. Detect and count the following:
    * Transformers
    * Telephone or junction boxes
6. Provide your findings in the following JSON format:

```json
{{
  "type": <pole_type>,
  "transformers": <number_of_transformers>,
  "telephone_or_junction_boxes": <number_of_telephone_or_junction_boxes>,
  "additional_notes": "<any_other_observations>"
}}
```

"""

print("Few-shot JSON examples and prompt reconstructed with updated fields.")

Few-shot JSON examples and prompt reconstructed with updated fields.


In [ ]:


if 'gcs_uris' in locals() and gcs_uris:
    for uri in gcs_uris:
        print(f"--- Classifying {uri} ---")
        classification = classify_image_with_gemini(uri, prompt)
        print(f"Result: {classification}\n")
else:
    print("No GCS URIs were found to classify.")

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:566314da74215735b48631b99f3b8429:00250082.jpg ---
Result: ```json
{
  "type": "utility pole",
  "transformers": 0,
  "telephone_or_junction_boxes": 0,
  "additional_notes": "This appears to be a street light pole."
}
```

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:01cdba18fbed09262fba4f92e67ddb00:00250082.jpg ---
Result: ```json
{
  "type": "utility pole",
  "transformers": 0,
  "telephone_or_junction_boxes": 0,
  "additional_notes": "The pole is a street light pole."
}
```

--- Classifying gs://geoai_published_ba3433e0-d709-4622-9577-66ab23587729__us/e8761cd3-c872-43cd-90aa-3ee99378090f/v0/o1:b83b2016a26a802b1ccb71c27753d65c:00250082.jpg ---
Result: ```json
{
  "type": "utility pole",
  "transformers": 0,
  "telephone_or_junction_boxes": 0,
  "additional_notes": "The pole appears to be pri